### Questions 
- nb de données != multiple de 24 ?
- date / date & heure de comptage pas redondant ? 
- qu'est-ce que la variable *filename* ? Nom du fichier depuis lequel on a importé les données ? (il n'y a que champs élysées & convention) 
- idem *etat de l'arc*

### Infos :
- *filename* plus utile

### TODO :
- [x] List variables that need to be predicted
- [x] Combine data of all 3 arcs
- [ ] Sort by to predict or not
- [ ] one hot encode string variables (ex: moon phases)
- encode dates : ideas :
    - Time since 1970 (continuous) ? **not sure**
    - Day of year in [1, 365] (categorical)
    - Month of year in [1, 12] (categorical)
    - Day of week in [1, 7] (categorical)
    - Is weekend (binary indicator)
    - Is holiday (binary indicator)
    - etc.

## imports

In [63]:
from utils import load_df
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses
import ipywidgets as ipw

## Load Data

In [129]:
train_df = load_df("data\\df_concat_train.pkl")
print(train_df.columns)
print(train_df['filename'].unique())

Index(['Date et heure de comptage', 'champs-elysees_Débit horaire',
       'champs-elysees_Taux d'occupation', 'champs-elysees_Etat trafic',
       'champs-elysees_Etat arc', 'convention_Débit horaire',
       'convention_Taux d'occupation', 'convention_Etat trafic',
       'convention_Etat arc', 'sts_Débit horaire', 'sts_Taux d'occupation',
       'sts_Etat trafic', 'sts_Etat arc', 'filename', 'Date',
       'Jour de la semaine_0', 'Jour de la semaine_1', 'Jour de la semaine_2',
       'Jour de la semaine_3', 'Jour de la semaine_4', 'Jour de la semaine_5',
       'Jour de la semaine_6', 'Etat du confinement', 'Couvre-feu',
       'Jour férié', 'Vacances scolaires',
       'Date des prochaines vacances scolaires',
       'Temps avant les prochaines vacances scolaires', 'tempC',
       'windspeedKmph', 'winddirDegree', 'weatherCode', 'precipMM', 'humidity',
       'visibility', 'pressure', 'cloudcover', 'HeatIndexC', 'DewPointC',
       'WindChillC', 'WindGustKmph', 'FeelsLikeC', 'uvInd

## Clean the data

In [99]:
def convert_types(df):
#     corrected_df = df.astype({: "float64"})
    return corrected_df


# temporary drops all strings and datetime
clean_train_df = train_df.drop(['filename', 'Date', 'Date et heure de comptage', 'moon_phase', 'champs-elysees_Etat arc', 'convention_Etat arc', 'sts_Etat arc', "Date des prochaines vacances scolaires", "Temps avant les prochaines vacances scolaires", "sunrise", "sunset"], axis=1)
# cc_train_df = convert_types(clean_train_df)

features_to_predict = ['tempC','windspeedKmph', 'winddirDegree', 'weatherCode', 'precipMM', 'humidity','visibility', 'pressure', 'cloudcover', 'HeatIndexC', 'DewPointC','WindChillC', 'WindGustKmph', 'FeelsLikeC', 'uvIndex', 'maxtempC','mintempC', 'avgtempC', 'totalSnow_cm', 'sunHour', 'uvIndex', 'moon_illumination', 'champs-elysees_Débit horaire', "champs-elysees_Taux d'occupation", 'champs-elysees_Etat trafic','convention_Débit horaire',"convention_Taux d'occupation", "convention_Etat trafic", 'sts_Débit horaire', "sts_Taux d'occupation", 'sts_Etat trafic']
nb_features = clean_train_df.shape[1]
nb_to_predict = len(features_to_predict)
nb_examples = clean_train_df.shape[0]
print("nb features : ", nb_features)
print("nb to predict : ", nb_to_predict)
print("nb examples : ", nb_examples)




def generate_table(df):
    print("| Nom       | Type | à prédire |")
    print("| :-- | :-- | :-: |")
    
    type_corresp = {"uint8":"entier", "int64":"entier", "bool": "booléen", "float64": "flottant", "datetime64[ns]": "datetime", "object": "chaîne de caractère", "timedelta64[ns]": "timedelta"}
    
    for x, y in zip(df.columns, df.dtypes):
        if str(x) in features_to_predict:
            print("|", x, "|", type_corresp[str(y)], "|",'<span style="color:green">oui</span>', "|")
        else:
            print("|", x, "|", type_corresp[str(y)], "|",'<span style="color:red">non</span>', "|")
# generate_table(clean_train_df)
# print(clean_train_df.dtypes)

nb features :  45
nb to predict :  31
nb examples :  9259


### Variables info

| Nom       | Type | à prédire |
| :-- | :-- | :-: |
| Date et heure de comptage | datetime | <span style="color:red">non</span> |
| champs-elysees_Débit horaire | flottant | <span style="color:green">oui</span> |
| champs-elysees_Taux d'occupation | flottant | <span style="color:green">oui</span> |
| champs-elysees_Etat trafic | entier | <span style="color:green">oui</span> |
| champs-elysees_Etat arc | chaîne de caractère | <span style="color:green">oui</span> |
| convention_Débit horaire | flottant | <span style="color:green">oui</span> |
| convention_Taux d'occupation | flottant | <span style="color:green">oui</span> |
| convention_Etat trafic | entier | <span style="color:green">oui</span> |
| convention_Etat arc | chaîne de caractère | <span style="color:green">oui</span> |
| sts_Débit horaire | flottant | <span style="color:green">oui</span> |
| sts_Taux d'occupation | flottant | <span style="color:green">oui</span> |
| sts_Etat trafic | entier | <span style="color:green">oui</span> |
| sts_Etat arc | chaîne de caractère | <span style="color:green">oui</span> |
| Jour de la semaine_0 | entier | <span style="color:red">non</span> |
| Jour de la semaine_1 | entier | <span style="color:red">non</span> |
| Jour de la semaine_2 | entier | <span style="color:red">non</span> |
| Jour de la semaine_3 | entier | <span style="color:red">non</span> |
| Jour de la semaine_4 | entier | <span style="color:red">non</span> |
| Jour de la semaine_5 | entier | <span style="color:red">non</span> |
| Jour de la semaine_6 | entier | <span style="color:red">non</span> |
| Etat du confinement | entier | <span style="color:red">non</span> |
| Couvre-feu | booléen | <span style="color:red">non</span> |
| Jour férié | booléen | <span style="color:red">non</span> |
| Vacances scolaires | booléen | <span style="color:red">non</span> |
| Date des prochaines vacances scolaires | datetime | <span style="color:red">non</span> |
| Temps avant les prochaines vacances scolaires | timedelta | <span style="color:red">non</span> |
| tempC | entier | <span style="color:green">oui</span> |
| windspeedKmph | entier | <span style="color:green">oui</span> |
| winddirDegree | entier | <span style="color:green">oui</span> |
| weatherCode | entier | <span style="color:green">oui</span> |
| precipMM | flottant | <span style="color:green">oui</span> |
| humidity | entier | <span style="color:green">oui</span> |
| visibility | entier | <span style="color:green">oui</span> |
| pressure | entier | <span style="color:green">oui</span> |
| cloudcover | entier | <span style="color:green">oui</span> |
| HeatIndexC | entier | <span style="color:green">oui</span> |
| DewPointC | entier | <span style="color:green">oui</span> |
| WindChillC | entier | <span style="color:green">oui</span> |
| WindGustKmph | entier | <span style="color:green">oui</span> |
| FeelsLikeC | entier | <span style="color:green">oui</span> |
| uvIndex | entier | <span style="color:green">oui</span> |
| uvIndex | entier | <span style="color:green">oui</span> |
| maxtempC | entier | <span style="color:green">oui</span> |
| mintempC | entier | <span style="color:green">oui</span> |
| avgtempC | entier | <span style="color:green">oui</span> |
| totalSnow_cm | flottant | <span style="color:green">oui</span> |
| sunHour | flottant | <span style="color:green">oui</span> |
| uvIndex | entier | <span style="color:green">oui</span> |
| uvIndex | entier | <span style="color:green">oui</span> |
| sunrise | datetime | <span style="color:green">oui</span> |
| sunset | datetime | <span style="color:green">oui</span> |
| moon_phase | chaîne de caractère | <span style="color:green">oui</span> |
| moon_illumination | entier | <span style="color:green">oui</span> |
| Journée | booléen | <span style="color:red">non</span> |

## Prepare data

#### Input data :
les points de données des instants `t-n` à `t`
- débit horaire [int]
- Taux d'occupation [float]
- météo : température, couverture nuageuse, ...

#### Output data :
for each arc (x3) instant `t+1`:
- débit horaire [int]
- Taux d'occupation [float]

<br/>
And all data that needs to be predicted 

In [124]:
def get_X_y(df, nb_steps=24):
    """ returns the X and y vectors from dataframe 
        steps : nb of datapoints in each Xi
    """ 
    nb_train_ex = nb_examples - (nb_steps + 1) + 1 # 1 in () : the y 
    print("nb train examples : ",nb_train_ex)
    print("nb features : ", nb_features)
    print("nb to predict : ", nb_to_predict)
    
    X = np.empty((nb_train_ex, nb_features, nb_steps))
    y = np.empty((nb_train_ex, nb_to_predict))
    for k_start in range(nb_examples):  
        X_k = clean_train_df.loc[k_start:k_start + nb_steps-1].to_numpy().reshape(nb_features, nb_steps)
        y_k = df[features_to_predict].iloc[k_start + nb_steps]
        print(df[features_to_predict].columns)
        print(features_to_predict)
        example_id = k_start
        print()
        print("y_k : ", y_k.shape)
        print("y npy : ", y[example_id, :].shape)
        X[example_id, :, :] = X_k
        y[example_id, :] = y_k
    return X, y


get_X_y(clean_train_df)
    
def display_predicted(X, y_pred):
    """ hey """
    pass

# print(clean_train_df[["Journée", "moon_illumination"]].iloc[24])
# fonction qui affiche à partir de Xi et yi (ou y_predi) les débits horaires et taux d'occupation

nb train examples :  9235
nb features :  45
nb to predict :  31
Index(['tempC', 'windspeedKmph', 'winddirDegree', 'weatherCode', 'precipMM',
       'humidity', 'visibility', 'pressure', 'cloudcover', 'HeatIndexC',
       'DewPointC', 'WindChillC', 'WindGustKmph', 'FeelsLikeC', 'uvIndex',
       'uvIndex', 'uvIndex', 'uvIndex', 'maxtempC', 'mintempC', 'avgtempC',
       'totalSnow_cm', 'sunHour', 'uvIndex', 'uvIndex', 'uvIndex', 'uvIndex',
       'moon_illumination', 'champs-elysees_Débit horaire',
       'champs-elysees_Taux d'occupation', 'champs-elysees_Etat trafic',
       'convention_Débit horaire', 'convention_Taux d'occupation',
       'convention_Etat trafic', 'sts_Débit horaire', 'sts_Taux d'occupation',
       'sts_Etat trafic'],
      dtype='object')
['tempC', 'windspeedKmph', 'winddirDegree', 'weatherCode', 'precipMM', 'humidity', 'visibility', 'pressure', 'cloudcover', 'HeatIndexC', 'DewPointC', 'WindChillC', 'WindGustKmph', 'FeelsLikeC', 'uvIndex', 'maxtempC', 'mintempC', 

ValueError: could not broadcast input array from shape (37) into shape (31)

## First model

In [ ]:
def model_1(input_shape, output_size=6, show_summary=False):
    model = models.Sequential()

    model.add(layers.Conv2D(128, kernel_size=(3, 3),
                        input_shape=input_shape, activation="relu"))
    model.add(layers.MaxPooling2D(3))
    model.add(layers.Dropout(0.2))

    model.add(layers.Conv2D(64, kernel_size=(3, 3), activation="relu"))
    model.add(layers.MaxPooling2D(3))
    model.add(layers.Dropout(0.2))

    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation="relu"))
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(output_size, activation="linear"))
    
    if show_summary:
        model.summary()
        
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    
    return model